<a href="https://colab.research.google.com/github/guilhermelaviola/IntelligentCommunication/blob/main/Class11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing all the necessary libraries:
import http.server
import json
import ssl
from urllib.parse import urlparse
import requests
from base64 import b64decode
from http import HTTPStatus

In [ ]:
# API configuration:
GOOGLE_API_KEY = 'YOUR_GOOGLE_API_KEY'  # Replace with your API key